#MYTCAS SCRAPING

In [1]:
import time
import json
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

def save_data_to_json(data, filename='all_universities_data.json'):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

options = Options()
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--headless')  # เพิ่มออปชันนี้เพื่อทำให้ Chrome ทำงานแบบ headless

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://course.mytcas.com/universities/')

SCROLL_PAUSE_TIME = 2

# Scroll down to load all universities
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

driver.execute_script("window.scrollTo(0, 0);")
time.sleep(SCROLL_PAUSE_TIME)

# Find all university links and store their hrefs
university_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/universities/')]")
universities = [{"name": uni.text, "url": uni.get_attribute('href')} for uni in university_links]

# Data structure to hold all the scraped data
all_universities_data = []

try:
    for university in universities:
        university_name = university["name"]
        university_url = university["url"]
        print(f"Clicking on {university_name}")

        driver.get(university_url)
        time.sleep(SCROLL_PAUSE_TIME)

        university_data = {
            "มหาวิทยาลัย": university_name,
            "รายละเอียด": {}
        }

        faculty_link = None
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            if re.search(r'วิศวกรรม', link.text):
                faculty_link = link
                break

        if faculty_link:
            driver.execute_script("arguments[0].scrollIntoView();", faculty_link)
            time.sleep(SCROLL_PAUSE_TIME)
            faculty_link.click()
        else:
            print("วิศวกรรม not found in", university_name)
            continue

        time.sleep(SCROLL_PAUSE_TIME)

        # Find all engineering branches (all links that contain 'fields')
        engineering_branches = driver.find_elements(By.XPATH, "//a[contains(@href, '/fields/')]")

        # Iterate over each branch URL
        for branch in engineering_branches:
            branch_name = branch.text
            branch_url = branch.get_attribute('href')
            print(f"  Clicking on {branch_name}")

            driver.get(branch_url)
            time.sleep(SCROLL_PAUSE_TIME)

            # Find all program links (หลักสูตร)
            program_links = driver.find_elements(By.XPATH, "//a[contains(@href, '/programs/')]")
            programs = []

            for program_link in program_links:
                program_name = program_link.find_element(By.CLASS_NAME, 'program').text
                program_type = program_link.find_element(By.CLASS_NAME, 'program-type').text
                program_url = program_link.get_attribute('href')
                print(f"    Visiting program: {program_name} {program_type}")

                driver.get(program_url)
                time.sleep(SCROLL_PAUSE_TIME)

                # Extract program details
                details = {}
                detail_elements = driver.find_elements(By.XPATH, "//dl//dt")
                for dt in detail_elements:
                    key = dt.text.strip()
                    dd = dt.find_element(By.XPATH, "following-sibling::dd")
                    value = dd.text.strip()
                    details[key] = value

                programs.append(details)

                driver.back()
                time.sleep(SCROLL_PAUSE_TIME)

            university_data["รายละเอียด"][branch_name] = programs

            # Navigate back to the Faculty of Engineering page
            driver.back()
            time.sleep(SCROLL_PAUSE_TIME)

        all_universities_data.append(university_data)

        # Navigate back to the universities list page
        driver.back()
        time.sleep(SCROLL_PAUSE_TIME)

        # Scroll down and up to ensure all elements are loaded properly
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(SCROLL_PAUSE_TIME)

except Exception as e:
    print(f"An error occurred: {e}")
    save_data_to_json(all_universities_data)
    driver.quit()
    raise

# Save the JSON data to a file
save_data_to_json(all_universities_data)

driver.quit()


Clicking on จุฬาลงกรณ์มหาวิทยาลัย
  Clicking on 1. วิศวกรรมทั่วไป
    Visiting program: หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมศาสตร์ (ภาษาไทย ปกติ)
  Clicking on 2. วิศวกรรมคอมพิวเตอร์
    Visiting program: หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอมพิวเตอร์ (ภาษาไทย ปกติ)
    Visiting program: หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอมพิวเตอร์และเทคโนโลยีดิจิทัล (ภาษาไทย ปกติ)
  Clicking on 3. วิศวกรรมเคมี
    Visiting program: หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมเคมีและกระบวนการ (หลักสูตรนานาชาติ) (นานาชาติ)
  Clicking on 4. วิศวกรรมนิวเคลียร์
    Visiting program: หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมนิวเคลียร์และรังสี (ภาษาไทย ปกติ)
  Clicking on 5. วิศวกรรมไฟฟ้า
    Visiting program: หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมไฟฟ้า (ภาษาไทย ปกติ)
  Clicking on 6. วิศวกรรมไฟฟ้าสื่อสาร วิศวกรรมโทรคมนาคม
    Visiting program: หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมสารสนเทศและการสื่อสาร (หลักสูตรนานาชาติ) (นานาชาติ)
  Clicking on 7. วิศวกรรมยานยนต์
